# Task 1
Get facility_code and relative features

In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

load_dotenv("API.env")
API_KEY = os.getenv("OE_API_KEY")
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

def fetch_facilities():
    """
    Retrieve the full list of facilities from OpenElectricity using pagination.
    The endpoint returns pages with a 'links.next' URL; we keep following 'next'
    until it is None. We also validate the Content-Type to ensure JSON is returned.
    Returns:
        list[dict]: Raw facility objects as returned by the API across all pages.
    """
    url = "https://api.openelectricity.org.au/v4/facilities/"
    facilities = []

    while url:
        # Single page request with a 30s timeout to avoid hanging.
        resp = requests.get(url, headers=HEADERS, timeout=30)

        # Defensive: ensure the server actually returned JSON before parsing.
        if not resp.headers.get("content-type", "").lower().startswith("application/json"):
            print("⚠️ Not JSON:", resp.status_code)
            print(resp.text[:300])  # print a short snippet to help debugging
            break

        # Raise an HTTPError if a non-2xx status code occurred (e.g., 401/403/500)
        resp.raise_for_status()

        # Parse JSON body; use empty dict fallback if somehow None
        j = resp.json() or {}

        # Append the current page's 'data' array to our running list
        facilities.extend(j.get("data", []))

        # Follow pagination: if 'links.next' exists, continue; else stop loop
        url = (j.get("links") or {}).get("next")
        if not url:
            break

    return facilities

# Call the fetcher once to retrieve all facilities
facilities = fetch_facilities()

def facilities_to_df(items):
    """
    Transform the raw facility objects into a tidy pandas DataFrame containing
    only the fields we need downstream. If a facility has multiple units, we
    read fuel technology from the first unit as a simple representative choice.
    Args:
        items (list[dict]): Raw facility JSON objects.
    Returns:
        pandas.DataFrame: Clean table with one row per facility_code.
    """
    rows = []
    for f in items:
        # If 'units' exists, take the first unit; otherwise fall back to empty dict.
        unit = (f.get("units") or [{}])[0]  # first generation unit if available

        # Build a flat record; use .get(..., None) style to avoid KeyError.
        rows.append({
            "facility_code": f.get("code"),
            "facility_name": f.get("name"),
            # Some schemas may expose 'region' instead of 'network_region';
            # this code prefers 'network_region' to match NEM naming like NSW1.
            "region": f.get("network_region"),
            "fuel_tech": unit.get("fueltech_id"),
            # Location is nested under 'location' with 'lat'/'lng' keys.
            "lat": (f.get("location") or {}).get("lat"),
            "lon": (f.get("location") or {}).get("lng"),
        })

    # Create a DataFrame and perform basic cleaning:
    # - drop rows missing facility_code (we need a stable key)
    # - drop duplicate facility_code rows
    # - sort by facility_code for deterministic output
    df = (pd.DataFrame(rows)
            .dropna(subset=["facility_code"])
            .drop_duplicates(subset=["facility_code"])
            .sort_values("facility_code")
            .reset_index(drop=True))
    return df

# Convert to a clean table and persist to CSV for later tasks
df_codes = facilities_to_df(facilities)

# Ensure the data directory exists before writing (optional safeguard)
os.makedirs("data", exist_ok=True)

# Save as UTF-8 CSV (no index column)
df_codes.to_csv("data/facilities.csv", index=False, encoding="utf-8")


1) Retrieve per-facility power and CO2 emissions (5m) for one week.
2) (Optional) Retrieve market price and demand (5m) by region.
3) Integrate timeseries into a single CSV for later MQTT streaming / dashboard.

In [ ]:
import os, json, math, time, requests
import pandas as pd
from math import ceil
from datetime import datetime, timedelta
from dotenv import load_dotenv

In [ ]:
# -------------------- Config (edit as needed) --------------------
WEEK_START = "2025-10-12T00:00:00"   # UTC, without trailing Z
WEEK_END   = "2025-10-18T23:59:59"
INTERVAL   = "5m"

BATCH_SIZE  = 30      # Number of facilities per request (30–50 is common)
CHUNK_HOURS = 2       # Initial chunk length; auto-shrinks on HTTP 416
MIN_HOURS   = 1       # Minimum chunk length (hours) for backoff
SLEEP_SEC   = 0.05    # Light rate limit between requests; increase on 429


DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)
OUT_CSV     = f"{DATA_DIR}/facility_timeseries_all.csv"
FACILITY_CSV= f"{DATA_DIR}/facilities.csv"

# -------------------- AUTH --------------------
# Load from API.env (optional) and environment. Required var: OE_API_KEY
load_dotenv("API.env")
API_KEY = os.getenv("OE_API_KEY", "").strip()

# If you prefer to hardcode (NOT recommended for submission), uncomment:
# API_KEY = "oe_xxx_your_token_here"

if not API_KEY:
    raise RuntimeError("Missing OE_API_KEY. Set it in environment or API.env.")

HEADERS = {"Authorization": f"Bearer {API_KEY}", "Accept": "application/json"}

# Endpoints
API_BASE_FAC = "https://api.openelectricity.org.au/v4/data/facilities/NEM"
API_FAC_LIST = "https://api.openelectricity.org.au/v4/facilities/"
API_BASE_MKT = "https://api.openelectricity.org.au/v4/market/network/NEM"

# -------------------- HTTP helpers --------------------
def http_get(url, params, headers=HEADERS, timeout=30, retry=2):
    """GET with small retry backoff for transient network issues."""
    for k in range(retry + 1):
        try:
            return requests.get(url, headers=headers, params=params, timeout=timeout)
        except requests.RequestException:
            if k < retry:
                time.sleep(0.8 * (k + 1))
            else:
                raise

# -------------------- Facilities master download --------------------
def ensure_facilities_csv(path=FACILITY_CSV):
    """Download facilities master list once and cache as CSV under data/."""
    if os.path.exists(path):
        return pd.read_csv(path)

    facilities, url = [], API_FAC_LIST
    while url:
        r = http_get(url, params={})
        r.raise_for_status()
        if "application/json" not in r.headers.get("content-type","").lower():
            raise RuntimeError(f"Unexpected content-type from {url}")
        j = r.json() or {}
        facilities.extend(j.get("data", []))
        url = (j.get("links") or {}).get("next")

    rows = []
    for f in facilities:
        rows.append({
            "facility_code": f.get("code"),
            "facility_name": f.get("name"),
            "region": f.get("region") or f.get("network_region"),
            "fuel_tech": f.get("fuel_tech") or f.get("fuel_tech_desc"),
        })

    df = (pd.DataFrame(rows)
            .dropna(subset=["facility_code"])
            .drop_duplicates(subset=["facility_code"])
            .sort_values("facility_code").reset_index(drop=True))
    df.to_csv(path, index=False, encoding="utf-8")
    print(f"[init] saved {path}, rows={len(df)}")
    return df

# -------------------- Time & batching helpers --------------------
def chunk_ranges(start_iso, end_iso, hours):
    """Yield [start,end] ISO pairs in fixed hour chunks (inclusive)."""
    s = datetime.fromisoformat(start_iso); e = datetime.fromisoformat(end_iso)
    cur = s
    while cur <= e:
        nxt = cur + timedelta(hours=hours) - timedelta(seconds=1)
        if nxt > e: nxt = e
        yield cur.strftime("%Y-%m-%dT%H:%M:%S"), nxt.strftime("%Y-%m-%dT%H:%M:%S")
        cur = cur + timedelta(hours=hours)

def batch_list(items, batch_size):
    for i in range(0, len(items), batch_size):
        yield items[i:i+batch_size]

# -------------------- Parsing helpers (robust to slight schema changes) --------------------
def _extract_fac_from_name(series_name: str, metric: str | None):
    """Try to infer facility_code from 'name' by stripping metric and network prefixes."""
    if not series_name:
        return None
    name = str(series_name)
    m = (metric or "").lower()
    low = name.lower()

    # remove common network prefixes
    for p in ("nem_", "wem_", "nem-", "wem-"):
        if low.startswith(p):
            name = name[len(p):]; low = low[len(p):]; break

    # strip metric as prefix or suffix with common separators
    for sep in ("_", "-", ":", "/", " "):
        if low.startswith(m + sep):
            name = name[len(m + sep):]; low = low[len(m + sep):]; break
        if low.endswith(sep + m):
            name = name[:-(len(sep + m))]; low = low[:-(len(sep + m))]; break

    if low in ("power", "emissions", ""):
        return None
    return name

def parse_results_robust(j, default_code=None):
    """Parse facility timeseries JSON into rows of {facility_code,timestamp,metric,value,unit}."""
    rows=[]
    for ts in (j.get("data") or []):
        metric = ts.get("metric")            # "power"/"emissions"
        unit   = ts.get("unit")
        results = ts.get("results") or []
        for res in results:
            if isinstance(res, dict):
                series_name = res.get("name") or ""
                datapoints  = res.get("data") or res.get("values") or []
                series_fc   = res.get("facility_code") or res.get("facility")
                if not series_fc:
                    series_fc = _extract_fac_from_name(series_name, metric)
            elif isinstance(res, (list, tuple)):
                series_name = ""
                datapoints  = res
                series_fc   = None
            else:
                continue

            for dp in datapoints:
                if isinstance(dp, dict):
                    ts0 = dp.get("timestamp") or dp.get("ts")
                    val = dp.get("value") or dp.get("v")
                    fc  = dp.get("facility_code") or dp.get("facility") or series_fc or default_code
                elif isinstance(dp, (list, tuple)) and len(dp) >= 2:
                    ts0, val = dp[0], dp[1]
                    fc = series_fc or default_code
                else:
                    continue

                if fc and str(fc).lower() not in ("power","emissions") and ts0 is not None and val is not None:
                    rows.append({
                        "facility_code": fc,
                        "timestamp": ts0,
                        "metric": metric,
                        "value": val,
                        "unit": unit
                    })
    return rows

def _extract_region_from_name(series_name: str, metric: str | None):
    """Extract NEM region (NSW1/QLD1/SA1/TAS1/VIC1) by removing metric and prefixes."""
    if not series_name:
        return None
    name = str(series_name)
    low = name.lower()
    m = (metric or "").lower()
    for p in ("nem_", "nem-"):
        if low.startswith(p):
            name = name[len(p):]; low = low[len(p):]; break
    for sep in ("_", "-", ":", "/", " "):
        if low.startswith(m + sep):
            name = name[len(m + sep):]; break
        if low.endswith(sep + m):
            name = name[:-(len(sep + m))]; break
    rg = name.strip()
    if rg.lower() in ("price","demand",""):
        return None
    return rg

def parse_market_json(j):
    """Parse /v4/market/network/NEM JSON to rows: {region,timestamp,metric,value,unit}."""
    rows=[]
    for ts in (j or {}).get("data") or []:
        metric = ts.get("metric")
        unit   = ts.get("unit")
        for res in (ts.get("results") or []):
            cols   = res.get("columns") or {}
            region = cols.get("region") or res.get("region")
            name   = res.get("name") or ""
            if not region:
                region = _extract_region_from_name(name, metric)
            pts = res.get("data") or res.get("values") or []
            for dp in pts:
                if isinstance(dp, dict):
                    ts0 = dp.get("timestamp") or dp.get("ts")
                    val = dp.get("value") or dp.get("v")
                elif isinstance(dp, (list,tuple)) and len(dp) >= 2:
                    ts0, val = dp[0], dp[1]
                else:
                    continue
                if region and ts0 is not None and val is not None:
                    rows.append({"region": region, "timestamp": ts0,
                                 "metric": metric, "value": val, "unit": unit})
    return rows

# -------------------- API fetchers --------------------
def _norm_region(x):
    if x is None:
        return None
    s = str(x).strip().upper()
    for p in ("NEM_", "NEM-", "NEM>", "NEM/"):
        if s.startswith(p):
            s = s[len(p):]
            break
    return s

def fetch_market_chunk(start_iso, end_iso):
    p = {
        "metrics": ["price","demand"],  # If 422 occurs, try "price,demand"
        "interval": INTERVAL,
        "date_start": start_iso,
        "date_end":   end_iso,
        "primary_grouping": "network_region",
    }
    r = http_get(API_BASE_MKT, params=p)
    if r.status_code == 416: return "416", []
    if r.status_code != 200 or "application/json" not in r.headers.get("content-type","").lower():
        return f"{r.status_code}", []
    return "OK", parse_market_json(r.json())

def fetch_facility_chunk(codes, start_iso, end_iso):
    p = {
        "metrics": ["power","emissions"],   # Fetch both metrics together
        "facility_code": list(codes),       # Batch facilities
        "interval": INTERVAL,
        "date_start": start_iso, "date_end": end_iso,
    }
    r = http_get(API_BASE_FAC, params=p)
    if r.status_code == 416: return "416", []
    if r.status_code != 200 or "application/json" not in r.headers.get("content-type","").lower():
        return f"{r.status_code}", []
    return "OK", parse_results_robust(r.json() or {})

# -------------------- Main: plan requests --------------------
df_fac_master = ensure_facilities_csv(FACILITY_CSV)
df_fac_master["region"] = df_fac_master["region"].apply(_norm_region)
all_codes = df_fac_master["facility_code"].dropna().astype(str).tolist()

print(f"[plan] FULL RUN facilities={len(all_codes)}")
start = datetime.fromisoformat(WEEK_START)
end   = datetime.fromisoformat(WEEK_END)
total_hours = int((end - start).total_seconds() // 3600) + 1
num_chunks  = ceil(total_hours / CHUNK_HOURS)
num_batches = ceil(len(all_codes) / BATCH_SIZE)
print(f"[estimate] facility requests ≈ {num_chunks * num_batches}")

# -------------------- Fetch facility timeseries (single run, no checkpoint) --------------------
rows_all = []
t0 = time.time()
for b_idx, batch in enumerate(batch_list(all_codes, BATCH_SIZE)):
    print(f"[batch {b_idx}] {batch[0]}.. (+{len(batch)-1} more)")
    for s_iso, e_iso in chunk_ranges(WEEK_START, WEEK_END, CHUNK_HOURS):
        cur_start, cur_end, cur_hours = s_iso, e_iso, CHUNK_HOURS
        while True:
            status, rows = fetch_facility_chunk(batch, cur_start, cur_end)
            if status == "OK":
                rows_all.extend(rows)
                break
            if status == "416" and cur_hours > MIN_HOURS:
                cur_hours = max(MIN_HOURS, cur_hours/2)
                seg_start = datetime.fromisoformat(cur_start)
                seg_end   = seg_start + timedelta(hours=cur_hours) - timedelta(seconds=1)
                cur_end   = seg_end.strftime("%Y-%m-%dT%H:%M:%S")
                print(f"[416] shrink → {cur_hours}h: {cur_start} ~ {cur_end}")
                continue
            print(f"[skip] {cur_start} ~ {cur_end} (status={status})")
            break
        time.sleep(SLEEP_SEC)

print(f"[done] facility raw points: {len(rows_all)} | elapsed={time.time()-t0:.1f}s")

# -------------------- Fetch market price/demand (optional) --------------------
mkt_rows_all = []
for s_iso, e_iso in chunk_ranges(WEEK_START, WEEK_END, CHUNK_HOURS):
    status, rows = fetch_market_chunk(s_iso, e_iso)
    if status == "OK":
        mkt_rows_all.extend(rows)
    elif status == "416":
        # Adaptive shrink on HTTP 416
        cur_start, cur_hours = s_iso, CHUNK_HOURS
        st = status
        while cur_hours > MIN_HOURS:
            cur_hours = max(MIN_HOURS, cur_hours/2)
            seg_end = (datetime.fromisoformat(cur_start)
                       + timedelta(hours=cur_hours) - timedelta(seconds=1)).strftime("%Y-%m-%dT%H:%M:%S")
            st, rows2 = fetch_market_chunk(cur_start, seg_end)
            if st == "OK":
                mkt_rows_all.extend(rows2); break
        if cur_hours == MIN_HOURS and st != "OK":
            print(f"[mkt skip] {s_iso} ~ {e_iso} status={status}")
    else:
        print(f"[mkt skip] {s_iso} ~ {e_iso} status={status}")

df_mkt_wide = pd.DataFrame()
if mkt_rows_all:
    dfm = pd.DataFrame(mkt_rows_all)
    dfm["timestamp"] = pd.to_datetime(dfm["timestamp"], utc=True)
    dfm["region"] = dfm["region"].apply(_norm_region)

    wstart = pd.Timestamp(WEEK_START + "Z"); wend = pd.Timestamp(WEEK_END + "Z")
    dfm = dfm[(dfm["timestamp"] >= wstart) & (dfm["timestamp"] <= wend)]

    df_mkt_wide = (dfm.pivot_table(index=["region","timestamp"],
                                   columns="metric", values="value", aggfunc="last")
                     .reset_index()
                     .rename(columns={"price":"price_aud_per_mwh","demand":"demand_mw"}))

# -------------------- Wide table + attributes + (optional) market merge --------------------
if rows_all:
    df_long = pd.DataFrame(rows_all)
    df_long["timestamp"] = pd.to_datetime(df_long["timestamp"], utc=True)

    wstart = pd.Timestamp(WEEK_START + "Z")
    wend   = pd.Timestamp(WEEK_END   + "Z")
    df_long = df_long[(df_long["timestamp"] >= wstart) & (df_long["timestamp"] <= wend)]

    df_wide = (df_long
        .pivot_table(index=["facility_code","timestamp"],
                     columns="metric", values="value", aggfunc="sum")
        .reset_index()
        .rename(columns={"power":"power_mw","emissions":"co2_kg"})
        .sort_values(["facility_code","timestamp"])
        .reset_index(drop=True)
    )

    # Facility attributes
    df_attr = df_fac_master[["facility_code","region","fuel_tech"]].copy()
    df_attr["region"] = df_attr["region"].apply(_norm_region)

    base = df_wide.merge(df_attr, on="facility_code", how="left")
    base["timestamp"] = pd.to_datetime(base["timestamp"], utc=True)
    base["region"]    = base["region"].apply(_norm_region)

    use_market = (
        isinstance(df_mkt_wide, pd.DataFrame)
        and not df_mkt_wide.empty
        and {"region", "timestamp"}.issubset(df_mkt_wide.columns)
    )

    if use_market:
        df_mkt_wide = df_mkt_wide.copy()
        df_mkt_wide["timestamp"] = pd.to_datetime(df_mkt_wide["timestamp"], utc=True)
        df_mkt_wide["region"]    = df_mkt_wide["region"].apply(_norm_region)

        df_out = (base.merge(df_mkt_wide, on=["region", "timestamp"], how="left")
                       .drop_duplicates(subset=["facility_code","timestamp"])
                       .sort_values(["facility_code","timestamp"])
                       .reset_index(drop=True))
    else:
        print("[mkt] empty or missing columns -> skip market merge")
        df_out = (base.drop_duplicates(subset=["facility_code","timestamp"])
                       .sort_values(["facility_code","timestamp"])
                       .reset_index(drop=True))

    # Save final integrated CSV
    df_out.to_csv(OUT_CSV, index=False)
    print(f"[save] {OUT_CSV} rows={len(df_out)}")
else:
    print("[warn] no facility timeseries rows collected; nothing saved.")


[plan] FULL RUN facilities=579
[estimate] facility requests ≈ 1680
[batch 0] 0BCWF.. (+29 more)
[batch 1] 0RIVERINASF.. (+29 more)
[batch 2] APS.. (+29 more)
[batch 3] BIALAWF.. (+29 more)
[batch 4] BUTLERSG.. (+29 more)
[batch 5] COLLIE_BESS2.. (+29 more)
[batch 6] DEIBDL.. (+29 more)
[batch 7] GIRGSF.. (+29 more)
[batch 8] HAYMSF.. (+29 more)
[batch 9] JUNEESF.. (+29 more)
[batch 10] LIMOSF2.. (+29 more)
[batch 11] METZSF.. (+29 more)
[batch 12] MUSSELRO.. (+29 more)
[batch 13] PHOENIX_KWINANA_WTE.. (+29 more)
[batch 14] ROWALLAN.. (+29 more)
[batch 15] SNOWY4.. (+29 more)
[batch 16] TARONGN.. (+29 more)
[batch 17] URANQ.. (+29 more)
[batch 18] WILLHOV.. (+29 more)
[batch 19] YAMBUK.. (+8 more)
[done] facility raw points: 1973434 | elapsed=995.0s
[save] data/facility_timeseries_all.csv rows=927922


# Task 2
Clean facilitiesl.csv

In [8]:
import pandas as pd

# Load the dataset
df = pd.read_csv("data/facilities.csv")

# Remove numeric suffix from region (e.g., QLD1 → QLD)
df["region"] = df["region"].str.replace(r"\d+$", "", regex=True)

# Clean facility_name: remove leading/trailing spaces
df["facility_name"] = df["facility_name"].str.strip()

# Standardize fuel_tech: lowercase and remove extra spaces
df["fuel_tech"] = df["fuel_tech"].str.lower().str.strip()

# Drop rows with missing latitude or longitude
df = df.dropna(subset=["lat", "lon"])

# Remove duplicated facility_code (keep the first occurrence)
df = df.drop_duplicates(subset="facility_code", keep="first")

# Export the cleaned dataset
df.to_csv("data/facilities.csv", index=False)


Clean facility_timeseries_all.csv

In [ ]:
# ===== Join & Clean: facilities × facility_timeseries =====
# Keep only matched facility_code records
# Remove QA columns, standardize region/fuel_tech, and convert timestamp

import pandas as pd

# -------------------- File Paths --------------------
FACILITIES_CSV = "data/facilities.csv"
TIMESERIES_CSV = "data/facility_timeseries_all.csv"
OUTPUT_CSV     = "data/cleaned_data.csv"

# -------------------- Load Data --------------------
fac = pd.read_csv(FACILITIES_CSV)
ts  = pd.read_csv(TIMESERIES_CSV, low_memory=False)

# -------------------- Basic Cleaning on facilities --------------------
# Remove trailing digits from region (QLD1 -> QLD)
fac["region"] = fac["region"].str.replace(r"\d+$", "", regex=True)

# Standardize text fields
fac["facility_code"] = fac["facility_code"].astype(str).str.strip()
fac["facility_name"] = fac["facility_name"].astype(str).str.strip()
fac["fuel_tech"] = fac["fuel_tech"].astype(str).str.lower().str.strip()

# -------------------- Inner Join (keep only matched facilities) --------------------
merged = ts.merge(fac, on="facility_code", how="inner", suffixes=("", "_fac"))

# -------------------- Fill Missing Fields --------------------
# Fill region and fuel_tech from facilities data
merged["region"] = merged["region"].fillna(merged["region_fac"])
merged["fuel_tech"] = merged["fuel_tech"].fillna(merged["fuel_tech_fac"])

# Remove redundant columns
merged = merged.drop(columns=["region_fac", "fuel_tech_fac"])

# -------------------- Convert Timestamp --------------------
merged["timestamp"] = pd.to_datetime(merged["timestamp"], utc=True, errors="coerce")
merged = merged.dropna(subset=["timestamp"])  # keep only valid timestamps

# -------------------- Save Cleaned File --------------------
merged.to_csv(OUTPUT_CSV, index=False)

# -------------------- Summary --------------------
print("✅ Join complete and cleaned.")
print(f"Rows: {len(merged):,}")
print(f"Columns: {merged.shape[1]}")
print("\nPreview:")
print(merged.head())


✅ Join complete and cleaned.
Rows: 161,731
Columns: 11

Preview:
  facility_code                 timestamp  co2_kg  power_mw region fuel_tech  \
0        AGLHAL 2025-10-12 00:00:00+00:00     0.0       0.0    SA1  gas_ocgt   
1        AGLHAL 2025-10-12 00:05:00+00:00     0.0       0.0    SA1  gas_ocgt   
2        AGLHAL 2025-10-12 00:10:00+00:00     0.0       0.0    SA1  gas_ocgt   
3        AGLHAL 2025-10-12 00:15:00+00:00     0.0       0.0    SA1  gas_ocgt   
4        AGLHAL 2025-10-12 00:20:00+00:00     0.0       0.0    SA1  gas_ocgt   

   demand_mw  price_aud_per_mwh facility_name       lat         lon  
0     340.15             -10.10       Hallett -33.34931  138.752633  
1     305.23             -10.10       Hallett -33.34931  138.752633  
2     324.03             -10.10       Hallett -33.34931  138.752633  
3     294.99              -9.22       Hallett -33.34931  138.752633  
4     250.23             -10.47       Hallett -33.34931  138.752633  


In [12]:
# ===== test2.csv Cleaning Script =====
# Actions:
# 1) Remove numeric suffix from region (e.g., QLD1 -> QLD)
# 2) Parse timestamp to UTC-aware datetime
# 3) Standardize text fields (fuel_tech lowercased, facility_name stripped)
# 4) For non-battery rows, clamp negative power_mw to 0
# 5) (Optional) Drop duplicate (facility_code, timestamp)
# 6) Save cleaned file and print a short summary

import pandas as pd
from pathlib import Path

# -------------------- Paths --------------------
INPUT_CSV  = "data/cleaned_data.csv"              # change path if needed
OUTPUT_CSV = "data/cleaned_data.csv"

# -------------------- Load --------------------
df = pd.read_csv(INPUT_CSV)

# -------------------- Region suffix removal --------------------
# e.g., 'QLD1' -> 'QLD', 'NSW1' -> 'NSW'
df["region"] = df["region"].astype("string").str.replace(r"\d+$", "", regex=True)

# -------------------- Timestamp parsing --------------------
# Convert to timezone-aware datetime (UTC). Invalid parses -> NaT
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce")
# Drop rows with invalid timestamps (rare, but keeps data consistent)
df = df.dropna(subset=["timestamp"])

# -------------------- Text standardization --------------------
# Keep fuel_tech normalized for robust filtering/grouping
df["fuel_tech"] = df["fuel_tech"].astype("string").str.lower().str.strip()
df["facility_name"] = df["facility_name"].astype("string").str.strip()

# -------------------- Power sanity handling --------------------
# For non-battery technologies, negative power often indicates anom./import.
# Business rule: set negative power to 0 only for NON-battery rows.
if "power_mw" in df.columns:
    non_batt_mask = ~df["fuel_tech"].fillna("").str.contains("battery", na=False)
    neg_mask = df["power_mw"] < 0
    df.loc[non_batt_mask & neg_mask, "power_mw"] = 0

# -------------------- (Optional) De-duplicate on key --------------------
# If you want to ensure unique time points per facility, enable next line:
# df = df.drop_duplicates(subset=["facility_code", "timestamp"], keep="first")

# -------------------- Save --------------------
df.to_csv(OUTPUT_CSV, index=False)

# -------------------- Summary --------------------
print("✅ Cleaning complete.")
print(f"Rows: {len(df):,} | Cols: {df.shape[1]}")
print("Time range:", df["timestamp"].min(), "→", df["timestamp"].max())

print("\nMissing rate (top 10):")
print(df.isna().mean().sort_values(ascending=False).head(10))

print("\nRegion sample counts:")
print(df["region"].value_counts().head(10))

print("\nFuel tech sample counts:")
print(df["fuel_tech"].value_counts().head(10))

print("\nPreview:")
print(df.head())


✅ Cleaning complete.
Rows: 161,731 | Cols: 11
Time range: 2025-10-12 00:00:00+00:00 → 2025-10-18 13:55:00+00:00

Missing rate (top 10):
price_aud_per_mwh    0.018494
demand_mw            0.006857
facility_code        0.000000
timestamp            0.000000
co2_kg               0.000000
power_mw             0.000000
region               0.000000
fuel_tech            0.000000
facility_name        0.000000
lat                  0.000000
dtype: float64

Region sample counts:
region
TAS    39759
NSW    37878
VIC    30304
QLD    27274
SA     26516
Name: count, dtype: Int64

Fuel tech sample counts:
fuel_tech
hydro                49244
wind                 41651
solar_utility        34092
gas_ccgt              9470
bioenergy_biomass     7561
gas_ocgt              5682
battery_charging      4616
distillate            3788
gas_wcmg              3733
battery               1894
Name: count, dtype: Int64

Preview:
  facility_code                 timestamp  co2_kg  power_mw region fuel_tech  \
0     